# Import Libraries

In [1]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
################ TIME MODLULES ###################
import time
from datetime import date 
import datetime
############## DATA MANIPULATION MODULES #########
import pandas as pd
import numpy as np
import re
import statistics 
from statistics import mode 

# Define Functions

In [2]:
def get_links_and_text():
    link=[i.get('href') for i in soup.find_all('a',{'style':'text-decoration:none;display:block'})]
    text=[i.text for i in soup.find_all('div',{'class':'JheGif nDgy9d'})]
    date=[i.text for i in soup.find_all('span',{'class':'WG9SHc'})]
    website=[i.text for i in soup.find_all('div',{'class':'XTjFC WF4CUc'})]
    ##################### FILTER VALUES ########################################
    INDEX_1=[i for i in range(len(text)) if '$' in text[i]]
    link=[link[i] for i in INDEX_1]
    text=[text[i] for i in INDEX_1]
    date=[date[i] for i in INDEX_1]
    website=[website[i] for i in INDEX_1]
    ############################################################################
    return link,text,date,website

In [3]:
def scrape_amount(z):
    key=['million','billion','millions','billions','trillion','trillions']
    amount=[]
    for x in z:
        try:
            if(x[x.index('$'):].split()[1].lower() in key):
                amount.append(x[x.index('$'):].split()[0]+' '+x[x.index('$'):].split()[1])
            else:
                amount.append(x[x.index('$'):].split()[0])
        except:
            amount.append(x[x.index('$'):].split()[0])
    return amount

In [4]:
def scrape_non_public(company):
    ticker=[]
    listed=[]
    public=[]
    for i in company:
        try:
            driver.get('https://www.google.com/search?tbm=fin&q=NASDAQ%3A%20{}'.format("+".join(i.split())))
            source = driver.page_source
            soup=bs4.BeautifulSoup(source, 'html.parser')
            ticker.append(soup.find('span',{'jsname':'qRSVye'}).text[0])
            listed.append(soup.find('div',{'class':'wx62f PZPZlf'}).text.split(':')[0])
            public.append(1)
        except:
            time.sleep(5)
            try:
                driver.get('https://www.google.com/search?tbm=fin&q={}'.format("+".join(i.split())))
                source = driver.page_source
                soup=bs4.BeautifulSoup(source, 'html.parser')
                ticker.append(soup.find('span',{'jsname':'qRSVye'}).text[0])
                listed.append(soup.find('div',{'class':'wx62f PZPZlf'}).text.split(':')[0])
                public.append(1)
            except:
                ticker.append(np.nan)
                listed.append(np.nan)
                public.append(0)
        time.sleep(5)
    return public,listed,ticker

# User Input

In [5]:
user=input('Do you want to enter companies one by one?(y/n): ')
l=[]
if(user=='y'):
    more='y'
    while(more=='y'):
        a=input('enter company name: ')
        l.append(a)
        more=input('do you want to enter more companies(y/n): ')
else:
    csv=input('enter name of excel file: ')
    companies=input('enter column containing list of companies: ')
    try:
        l=list(pd.read_csv(csv)[companies])
    except:
        l=list(pd.read_excel(csv)[companies])

In [8]:
# Taking 10/424 companies for example
l=l[:10]
l

['Bisichem',
 'Sinovac Biotech Co. Ltd.',
 'FLX Bio, Inc.',
 'Confo Therapeutics',
 'NeuroCycle Therapeutics, Inc.',
 'Protera ',
 'Swiss Medical Union SA',
 'Positrigo AG',
 'Aspect Biosystems',
 'deepCDR Biologics AG']

# Scrape [Google News](https://news.google.com/topstories?hl=en-IN&gl=IN&ceid=IN:en)

In [9]:
x,k,y,z,m=[],[],[],[],[]
driver = webdriver.Chrome(ChromeDriverManager().install()) # open Chrome driver/window
for i in l:
    print(i) # print company being scraped
    print('#'*45)
    # Scrape first 5 pages of "<COMPANY NAME> raises financing"
    for j in range(0,50,10): 
        driver.get('https://www.google.com/search?&tbm=nws&q={}+raises+financing&start={}'.format("+".join(i.split()),j))
        source = driver.page_source
        soup=bs4.BeautifulSoup(source, 'html.parser')
        link,text,date,website=get_links_and_text()
        print('Page No.',int(str(j)[0])+1)
        print('No. of links and text are: ',len(link),len(text),len(date),len(website))
        x.append(date)
        y.append(link)
        z.append(text)
        k.append(website)
        m.append([i for j in range(len(website))])
        time.sleep(5)
        print('*'*50)
    print('*'*100)

Bisichem
#############################################
Page No. 1
No. of links and text are:  0 0 0 0
**************************************************
Page No. 2
No. of links and text are:  0 0 0 0
**************************************************
Page No. 3
No. of links and text are:  0 0 0 0
**************************************************
Page No. 4
No. of links and text are:  0 0 0 0
**************************************************
Page No. 5
No. of links and text are:  0 0 0 0
**************************************************
****************************************************************************************************
Sinovac Biotech Co. Ltd.
#############################################
Page No. 1
No. of links and text are:  2 2 2 2
**************************************************
Page No. 2
No. of links and text are:  2 2 2 2
**************************************************
Page No. 3
No. of links and text are:  0 0 0 0
****************************************

In [10]:
# Normalize features
x=[j for i in x for j in i]
k=[j for i in k for j in i]
y=[j for i in y for j in i]
z=[j for i in z for j in i]
m=[j for i in m for j in i]
# Get amount from title
amount=scrape_amount(z)

In [35]:
# Create DataFrame from scraped features
df=pd.DataFrame(zip(m,x,z,y,amount),
                columns=['Company Name','Date','Article Title','URL','Amount'])

In [ ]:
# df.head()

In [38]:
df.shape

(59, 5)

In [18]:
# Keep only those rows where Company Name is in Article Title
df['temp']=df['Company Name'].apply(lambda x: x.split()[0])
df['C'] = df.apply(lambda x: str(x['temp']) in str(x['Article Title']), axis=1)
df=df.loc[df['C']==True].drop(['temp','C'],axis=1)

In [19]:
df.shape

(5, 5)

In [21]:
# Parse each article in DataFrame and scrape investors and series type
investors=[]
series=[]
for j,url in enumerate(df['URL']):
    print('Article',j)
    driver.get(url)
    source = driver.page_source
    soup=bs4.BeautifulSoup(source, 'html.parser')
    for script in soup(["script", "style"]):
        script.extract()    
    a = soup.get_text()
    a=" ".join(a.split())
    a=re.sub('[^A-Za-z0-9.]+', ' ', a)
    investors.append([i for i in a.split('.') if bool(re.search('investors ',i.lower()))==True])
    series.append([i.lower()[i.lower().find('series'):i.lower().find('series')+8] for i in a.split('.') if i.lower().find('series')!=-1])

Article 0
Article 1
Article 2
Article 3
Article 4


In [22]:
# Normalize investors and series type lists
series2=[]
def most_frequent(List): 
    return max(set(List), key = List.count) 

for i in range(len(investors)):
    if(investors[i]==[]):
        investors[i]=np.nan

for i in series:
    try:
        series2.append(most_frequent(i))
    except:
        series2.append(np.nan)

In [23]:
# Save lists to DataFrame
df['Series Type']=series2
df['Investors']=investors

In [25]:
# Make all values NaN in column "Series Type", except those which are in list allowed_vals
allowed_vals=['series b', 'series d', 'series a', 'series c','series e']
df['Series Type'][~df['Series Type'].isin(allowed_vals)] = np.nan

In [26]:
# Arranging Columns of DataFrame
df=df[['Company Name', 'Date', 'Article Title', 'URL', 'Series Type', 'Amount',
       'Investors']]

In [ ]:
# df

In [29]:
df.shape

(5, 7)

# Specify Date Range

In [28]:
try:
    d_range=input('Do you want for a date range(y/n): ')
    if(d_range=='y'):
        date1 = input('From(YYYY-MM-DD): ')
        date2 = input('To(YYYY-MM-DD): ')
        dates=[]
        dates2=[]
        start = datetime.datetime.strptime(date1, '%Y-%m-%d')
        end = datetime.datetime.strptime(date2, '%Y-%m-%d')
        step = datetime.timedelta(days=1)
    ################################################################
        while start <= end:
            dates.append (str(start.date()))
            start += step
        for i in dates:
            a1=i.split('-')
            x = datetime.datetime(int(a1[0]),int(a1[1]),int(a1[2]))
            dates2.append(x.strftime("%b %d, %Y"))
    ##################################################################
        df=df.loc[df['Date'].isin(dates2)]
except:
    pass

Do you want for a date range(y/n): n


# Adding not-scraped companies to DataFrame

In [30]:
k=df['Company Name'].unique()
not_scraped=list(set(l)-set(k))
for i in not_scraped:
    s2 = pd.DataFrame([i,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]).T
    s2.columns=df.columns
    df=df.append(s2)

In [31]:
df.shape

(12, 7)

# Sheet 2

In [32]:
df2=df.groupby(['Company Name']).size().reset_index(name='Value')
df2['Financing Rounds (Y/N)']=['Y' for i in range(df2.shape[0])]
for i in range(df2.shape[0]):
    if(df2['Company Name'][i] in not_scraped):
        df2['Value'][i]=0
        df2['Financing Rounds (Y/N)'][i]='N'

C:\Users\Amit\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Amit\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
df2

,Company Name,Value,Financing Rounds (Y/N)
0,Aspect Biosystems,2,Y
1,Bisichem,0,N
2,Confo Therapeutics,0,N
3,"FLX Bio, Inc.",0,N
4,"NeuroCycle Therapeutics, Inc.",0,N
5,Positrigo AG,0,N
6,Protera,1,Y
7,Sinovac Biotech Co. Ltd.,2,Y
8,Swiss Medical Union SA,0,N
9,deepCDR Biologics AG,0,N


# Save Outputs

In [34]:
from datetime import date
name='BiosectRx Financing Harvest-'+str(date.today())+'.xlsx'
writer = pd.ExcelWriter(name, engine='xlsxwriter')
df2.to_excel(writer, sheet_name='Financing Check',index=False)
df.to_excel(writer, sheet_name='Harvest',index=False)
writer.save()
driver.close()